Nesta alternativa, resolvi criar matrizes marvok separados para cada agrupamento.

Pensei em algo do tipo.

OLEO = MATRIZ_PRECO + MATRIZ_EXPORTACAO + MATRIZ_PRODUCAO

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Math #para escrever equações 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error
import statsmodels.api as sm
import seaborn as sns

import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

In [3]:
from random import randint
from xmlrpc.client import Boolean
from tqdm import tqdm
import pandas as pd
import numpy as np
import quantecon as qe

import logging
log = logging.getLogger(__name__)
# logging.basicConfig(level=logging.DEBUG) 


In [4]:
RANDOM_SIZE   = 1000000
RANDOM_SIZE   = 10000
QTD_BINS      = 3
TEST_SIZE     = 0.05
RANDOM_STATE  = 2022
PREDITOR                 = ['vegetable-oil_price', 'vegetable-oil_production', 'vegetable-oil_exports']
PREDITOR_DIRECTIONS      = ['vegetable-oil_price', 'vegetable-oil_production', 'vegetable-oil_exports']


- https://raw.githubusercontent.com/fkfouri/vegetable_oil_mkt/master/dataset/__by_month_commodity_types.csv
- https://raw.githubusercontent.com/fkfouri/vegetable_oil_mkt/master/dataset/__by_month_vegetable_oil.csv
- https://raw.githubusercontent.com/fkfouri/vegetable_oil_mkt/master/dataset/__by_month_full.csv

# Mercado de Oleo

## Dataset Completo
- USDA por comodity
- USDA Agrupado por Oleo Vegetal
- OECD Agrupado
- FAO Index Price


In [5]:
def get_dataset():
    df = pd.read_csv('https://raw.githubusercontent.com/fkfouri/vegetable_oil_mkt/master/dataset/__by_month_full.csv')
    print(df.shape)
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by=['date'], ascending=[True], inplace=True)
    df.set_index(['date'], inplace=True)
    return df

%time df_full = get_dataset()
df_full.head()

(266, 139)
CPU times: total: 78.1 ms
Wall time: 428 ms


,reference,coconut_beginning-stocks,coconut_crush,coconut_domestic-consumption,coconut_ending-stocks,coconut_exports,coconut_feed-waste-dom-cons,coconut_food-use-dom-cons,coconut_imports,coconut_industrial-dom-cons,...,vegetable-oil_total-supply,vegetable-oil_oecd_consumption,vegetable-oil_oecd_ending-stocks,vegetable-oil_oecd_exports,vegetable-oil_oecd_food,vegetable-oil_oecd_food-fat-availability,vegetable-oil_oecd_human-consumption-per-capita,vegetable-oil_oecd_imports,vegetable-oil_oecd_other-use,vegetable-oil_oecd_production
date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7273.770,844.920,2541.320,6179.990,2.240,1.020,2490.870,1088.770,7469.760
2000-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7298.970,849.930,2562.660,6190.320,2.240,1.020,2504.980,1098.620,7494.420
2000-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7324.160,854.950,2584.010,6200.660,2.240,1.020,2519.100,1108.460,7519.090
2000-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7349.360,859.960,2605.360,6211.000,2.240,1.030,2533.210,1118.310,7543.750
2000-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7374.550,864.970,2626.710,6221.340,2.250,1.030,2547.320,1128.150,7568.420


In [6]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 266 entries, 2000-01-31 to 2022-02-28
Columns: 138 entries, reference to vegetable-oil_oecd_production
dtypes: float64(137), object(1)
memory usage: 288.9+ KB


In [7]:
df_full.dtypes

reference                                           object
coconut_beginning-stocks                           float64
coconut_crush                                      float64
coconut_domestic-consumption                       float64
coconut_ending-stocks                              float64
                                                    ...   
vegetable-oil_oecd_food-fat-availability           float64
vegetable-oil_oecd_human-consumption-per-capita    float64
vegetable-oil_oecd_imports                         float64
vegetable-oil_oecd_other-use                       float64
vegetable-oil_oecd_production                      float64
Length: 138, dtype: object

In [8]:
#Identificacao de colunas/atributos
features = [x for x in df_full.columns if ('exports' in x or 'production' in x or 'price' in x) and not ('olive' in x or 'cottonseed' in x) ]
prices   = [x for x in df_full.columns if 'price' in x and 'groundnut' not in x  ]

print(f'features: {len(features)} | {features}')
print("==" * 30)
print(f'prices: {len(prices)} | {prices}')

features: 26 | ['coconut_exports', 'coconut_price', 'coconut_production', 'palm-kernel_exports', 'palm-kernel_price', 'palm-kernel_production', 'palm_exports', 'palm_price', 'palm_production', 'peanut_exports', 'peanut_price', 'peanut_production', 'rapeseed_exports', 'rapeseed_price', 'rapeseed_production', 'soybean_exports', 'soybean_price', 'soybean_production', 'sunflower_exports', 'sunflower_price', 'sunflower_production', 'vegetable-oil_exports', 'vegetable-oil_price', 'vegetable-oil_production', 'vegetable-oil_oecd_exports', 'vegetable-oil_oecd_production']
prices: 8 | ['coconut_price', 'palm-kernel_price', 'palm_price', 'peanut_price', 'rapeseed_price', 'soybean_price', 'sunflower_price', 'vegetable-oil_price']


In [9]:
#REMOÇÃO DE CAMPOS NAN
df = df_full[features].copy()
print(df.shape)
df = df[df['sunflower_production'].notnull()]
df = df[df['sunflower_price'].notnull()]
print(df.shape)
df

(266, 26)
(234, 26)


,coconut_exports,coconut_price,coconut_production,palm-kernel_exports,palm-kernel_price,palm-kernel_production,palm_exports,palm_price,palm_production,peanut_exports,...,soybean_price,soybean_production,sunflower_exports,sunflower_price,sunflower_production,vegetable-oil_exports,vegetable-oil_price,vegetable-oil_production,vegetable-oil_oecd_exports,vegetable-oil_oecd_production
date,,,,,,,,,,,,,,,,,,,,,
2002-02-28,148.440,376.000,255.940,130.470,356.000,264.710,1527.920,352.980,2146.310,17.030,...,364.920,3204.320,166.180,578.000,620.920,3312.180,47.786,8559.630,2986.310,7954.130
2002-03-31,148.040,366.000,255.830,130.830,353.240,266.520,1543.880,359.000,2162.830,16.580,...,359.210,3222.400,168.850,557.000,625.580,3339.800,47.622,8591.720,3014.280,7983.170
2002-04-30,147.640,411.000,255.720,131.190,405.000,268.330,1559.830,360.070,2179.360,16.140,...,370.860,3240.470,171.530,552.000,630.250,3367.440,48.969,8623.810,3042.260,8012.210
2002-05-31,147.240,420.000,255.610,131.560,418.300,270.150,1575.790,384.910,2195.890,15.690,...,399.890,3258.550,174.200,574.000,634.920,3395.080,51.641,8655.950,3070.240,8041.250
2002-06-30,146.830,446.000,255.500,131.920,439.000,271.960,1591.750,426.130,2212.420,15.250,...,456.810,3276.620,176.880,595.000,639.580,3422.710,55.804,8688.040,3098.220,8070.290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,157.080,1493.640,290.610,276.780,1341.140,716.470,4092.220,1141.820,6229.440,35.640,...,1433.940,6354.170,988.220,1355.691,1703.690,7821.270,165.862,18894.320,7325.470,18174.560
2021-09-30,158.080,1485.000,291.080,278.380,1427.270,718.600,4103.190,1181.380,6246.960,35.020,...,1398.750,6350.750,994.670,1309.519,1717.600,7837.790,168.571,18923.870,7328.510,18207.820
2021-10-31,159.080,1922.860,291.560,279.970,1818.330,720.740,4114.150,1310.250,6264.470,34.400,...,1483.520,6347.330,1001.110,1420.528,1731.510,7854.300,184.837,18953.420,7331.560,18241.070


In [10]:
df.describe()

,coconut_exports,coconut_price,coconut_production,palm-kernel_exports,palm-kernel_price,palm-kernel_production,palm_exports,palm_price,palm_production,peanut_exports,...,soybean_price,soybean_production,sunflower_exports,sunflower_price,sunflower_production,vegetable-oil_exports,vegetable-oil_price,vegetable-oil_production,vegetable-oil_oecd_exports,vegetable-oil_oecd_production
count,234.000,234.000,234.000,234.000,234.000,234.000,234.000,234.000,234.000,234.000,...,234.000,234.000,234.000,234.000,234.000,234.000,234.000,234.000,234.000,234.000
mean,154.793,1037.208,277.760,233.840,956.092,523.365,3226.635,763.075,4406.418,19.901,...,882.535,4857.446,563.021,966.043,1174.819,5990.575,103.046,14277.728,5545.769,13471.372
std,10.334,412.612,12.313,41.691,378.930,139.269,805.918,244.906,1262.844,6.470,...,272.357,980.073,284.888,334.951,331.668,1329.965,32.741,3166.837,1307.176,3118.232
min,123.000,366.000,248.500,130.470,353.240,264.710,1527.920,352.980,2146.310,12.830,...,359.210,3204.320,166.180,543.000,620.920,3312.180,47.622,8559.630,2986.310,7954.130
25%,148.055,702.250,270.998,219.213,655.290,399.255,2515.020,577.495,3275.315,15.468,...,719.372,4057.825,329.412,719.643,886.732,5027.562,78.436,11538.903,4463.065,10815.903
50%,154.895,953.500,275.410,245.990,878.330,530.115,3399.860,732.000,4468.605,16.805,...,832.990,4667.785,493.780,856.470,1127.385,5927.510,97.910,14325.960,5663.720,13418.630
75%,160.400,1387.287,287.767,263.450,1232.705,642.967,4014.682,886.188,5503.118,23.133,...,1016.713,5831.602,867.900,1174.505,1528.378,7278.573,121.403,17158.577,6772.868,16394.708
max,177.920,2256.000,302.500,283.170,2307.630,725.000,4325.500,1377.220,6299.500,40.580,...,1574.670,6381.500,1122.670,2045.000,1761.580,7887.340,184.837,19012.500,7349.260,18307.580


## Correlação

Paired density and scatterplot matrix

https://seaborn.pydata.org/examples/pair_grid_with_kde.html

In [11]:
if False:
    %time
    sns.set_theme(style="white")

    g = sns.PairGrid(df, diag_sharey=False)
    g.map_upper(sns.scatterplot, s=15)
    g.map_lower(sns.kdeplot)
    g.map_diag(sns.kdeplot, lw=2)

![Alt text](https://raw.githubusercontent.com/fkfouri/vegetable_oil_mkt/master/images/Compare.png)

## Dendograma

In [12]:
if False:
    plt.figure(figsize=(25,10))
    dendrogram = sch.dendrogram(sch.linkage(df, method  = "ward"))

    plt.title('Dendrogram')

    # plt.xlabel('Customers')
    plt.ylabel('Euclidean distances')
    %time plt.show()

# Markov
Observaçoes:
- Sera que o outcome deveria ser calculado em funcao do dado? Não deveria ser uma referencia Exogena? Exemplos (Open Interest da soja eu tenho). No exemplo do VIRAL... ele pega o outcome como o volume de negocios.
- O que fazer com os preditores dentro do random_set (??)

Tentativa de segregar os patterns em, para ter padroes menores:
- product_variation
- exports_variation
- price_variation



In [13]:
# !pip install --upgrade quantecon numba tqdm --quiet

## Library

In [14]:
get_divison  = lambda size: [item / size for item in list(range(size + 1))]
get_labels   = lambda size, start=0: [f'{chr( (item % 26 + start) + 65  )}{ "" if item // 26 <= 0 else item // 26 }' for item in list(range(size))]
get_full     = lambda size: [ (chr(i + 65), f'{j / size} < x <= { (j+1) / size}' ) for i , j in enumerate(range(size))]
get_bins     = lambda _input_array, size:  np.round(np.linspace(_input_array.min(), _input_array.max(), size + 1), 6).tolist()


def read_kwarg_list(kwargs, name, _else_conditions = None):
    if name in kwargs and isinstance(kwargs[name], list):
        return kwargs[name]
    return _else_conditions

def read_kwarg_bool(kwargs, name):
    if name in kwargs and isinstance(kwargs[name], Boolean):
        return kwargs[name]
    return False
        
    
def reset_logging():
    """
    Reset logging - Tira as definicoes de DEBUG, INFO, etc
    """
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
        

In [15]:
def convet_collection_to_dataframe(collection):
    """
    Concatena todas as colecoes
    Remove os np.inf e np.nan, gerados pelo pc_change
    """
    df_out = pd.concat(collection)
    
    log.debug(f'Before dropping NaNs: {df_out.shape}')
    df_out.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_out = df_out.dropna(how='any') 
    log.debug(f'After dropping NaNs: {df_out.shape}')
    return df_out

## Geração do Random Sets

In [16]:
# logging.basicConfig(level=logging.DEBUG) 

In [17]:
#Gera uma serie de forma randomica. Pega amostras do dataset com horizonte de 3 a 18 linhas (meses) e 
def get_random_sets(input_dataframe: pd.DataFrame, 
                    size = 100000,
                    **kwargs):
    
    # take random sets of sequential rows 
    new_set = []

    # breakpoint()
    
    log.debug(f'AQUI:   {size}')
    for row_set in (tqdm(iterable = range(0, size), bar_format='{desc:<15}{percentage:3.0f}%|{bar:50}{r_bar}' ) ):
        
        # obtem de forma randomica uma amostra de n linhas do dataset entre 3 e 18 meses
        row_quant     = randint(3, 18)
        row_start     = randint(0, len(input_dataframe) - row_quant)
        row_finish    = row_start + row_quant

        market_subset = input_dataframe.iloc[row_start:row_finish]
        Close_Date    = max(market_subset['date'])
        
        log.debug(f'AQUI:   {row_start} |{row_finish} | {market_subset.shape} | {Close_Date}')

        if row_set%(size//5)==0:
            pass
            print(f'row_set: {row_set:<6} | row_quant: {row_quant:2} | rows: {row_start:>6}-{row_finish-1:<6} | market_subset: {str(market_subset.shape):^10} | Close_Date: {Close_Date:%m/%d/%Y}')
               
        ref = {}
        ref['Sequence_ID']  = [row_set] * row_quant
        ref['Close_Date']   = [Close_Date] * row_quant
        ref['ref_date']     = market_subset['date']
        ref['size_dataset'] = [len(market_subset) - 2] * row_quant  # subtracao de dois devido limpeza NaN
        
#         # TODO: Leitura dos Preditores, Tipo Media do período  
#         # parece ser irrelevante... pois houve uma interpolcao durante upstream. 
#         # A Media tente a ser a mesma para datasets dentro de um mesmo ano
#         for name, value in market_subset[PREDITOR].iteritems():
#             ref[f'{name}_mean']       = value.mean() 
        

        # Colunas que NAO desejo realizar a variacao
        bypass_ref = {}
        
        if 'bypass' in kwargs and isinstance(kwargs['bypass'], list):
            for _col in kwargs['bypass']:
                bypass_ref[_col] = market_subset[_col]
                
                       
        # columnas que desejo ter as variacoes em relacao a linha anterior (date)
        columns_ref = {}
        if 'columns' in kwargs and isinstance(kwargs['columns'], list):
            for _col in kwargs['columns']:
                if _col in input_dataframe.columns and _col not in bypass_ref:
                    columns_ref[_col] = market_subset[_col]
                    columns_ref[f'{_col}_variation'] = market_subset[_col].pct_change()
                    
                    if 'outcomes' in kwargs and isinstance(kwargs['outcomes'], list) and _col in kwargs['outcomes']:
                        columns_ref[f'{_col}_outcome'] = market_subset[_col].diff(-1) * -1
        

               
        #merge dictionaries
        ref = {**ref, **bypass_ref, **columns_ref}
        
        new_set.append(pd.DataFrame(ref))
        
    return new_set

# get_random_sets(df.reset_index(), RANDOM_SIZE, columns=features,  outcomes=features, bypass=PREDITOR)

In [18]:
%time my_set_case2 = get_random_sets(df.reset_index(), RANDOM_SIZE, columns=features,  outcomes=features, bypass=[])

                 0%|                                                  | 10/10000 [00:00<03:49, 43.57it/s]

row_set: 0      | row_quant:  5 | rows:      7-11     | market_subset:  (5, 27)   | Close_Date: 06/30/2003


                20%|██████████                                        | 2009/10000 [00:30<01:57, 67.91it/s]

row_set: 2000   | row_quant: 18 | rows:    103-120    | market_subset:  (18, 27)  | Close_Date: 07/31/2012


                40%|████████████████████                              | 4008/10000 [01:02<02:36, 38.34it/s]

...

row_set: 6000   | row_quant: 13 | rows:     52-64     | market_subset:  (13, 27)  | Close_Date: 11/30/2007


                80%|████████████████████████████████████████          | 8012/10000 [02:09<00:30, 64.46it/s]

row_set: 8000   | row_quant: 15 | rows:     38-52     | market_subset:  (15, 27)  | Close_Date: 11/30/2006


               100%|██████████████████████████████████████████████████| 10000/10000 [02:40<00:00, 62.38it/s]

CPU times: total: 2min 42s
Wall time: 2min 40s


In [19]:
%time df1 = convet_collection_to_dataframe(my_set_case2)
print(df1.shape)
# df1.sort_values(by=['ref_date'])
df1

CPU times: total: 1.22 s
Wall time: 1.23 s
(82688, 82)


,Sequence_ID,Close_Date,ref_date,size_dataset,coconut_exports,coconut_exports_variation,coconut_exports_outcome,coconut_price,coconut_price_variation,coconut_price_outcome,...,vegetable-oil_price_outcome,vegetable-oil_production,vegetable-oil_production_variation,vegetable-oil_production_outcome,vegetable-oil_oecd_exports,vegetable-oil_oecd_exports_variation,vegetable-oil_oecd_exports_outcome,vegetable-oil_oecd_production,vegetable-oil_oecd_production_variation,vegetable-oil_oecd_production_outcome
8,0,2003-06-30,2003-03-31,3,145.500,0.002,0.360,441.000,-0.075,-20.000,...,0.333,9025.170,0.005,48.160,3314.130,0.005,16.010,8371.450,0.005,42.310
9,0,2003-06-30,2003-04-30,3,145.860,0.002,0.360,421.000,-0.045,19.000,...,1.525,9073.330,0.005,48.180,3330.140,0.005,16.010,8413.760,0.005,42.310
10,0,2003-06-30,2003-05-31,3,146.220,0.002,0.360,440.000,0.045,19.000,...,0.863,9121.510,0.005,48.170,3346.150,0.005,16.010,8456.070,0.005,42.300
106,1,2012-08-31,2011-05-31,15,152.170,0.003,0.410,2097.000,0.004,-294.000,...,-1.627,13874.250,0.004,50.330,5416.420,0.004,23.310,12981.450,0.004,46.890
107,1,2012-08-31,2011-06-30,15,152.580,0.003,0.420,1803.000,-0.140,-141.000,...,-4.723,13924.580,0.004,50.350,5439.730,0.004,23.310,13028.340,0.004,46.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,9999,2018-03-31,2017-10-31,12,147.380,-0.001,-0.150,1484.770,-0.073,69.550,...,1.176,17706.980,0.004,66.470,6872.920,0.002,12.130,17090.430,0.005,84.330
184,9999,2018-03-31,2017-11-30,12,147.230,-0.001,-0.150,1554.320,0.047,-103.660,...,-5.890,17773.450,0.004,66.470,6885.050,0.002,12.130,17174.760,0.005,84.330
185,9999,2018-03-31,2017-12-31,12,147.080,-0.001,2.570,1450.660,-0.067,-56.800,...,0.289,17839.920,0.004,42.950,6897.180,0.002,37.670,17259.090,0.005,31.120
186,9999,2018-03-31,2018-01-31,12,149.650,0.017,2.570,1393.860,-0.039,-149.610,...,-2.651,17882.870,0.002,42.950,6934.850,0.005,37.680,17290.210,0.002,31.110


In [20]:
# df1[PREDITOR]

In [21]:
#Lista de features tipo outcome
outcomes = [x for x in df1.columns if 'outcome' in x  ]
outcomes

['coconut_exports_outcome',
 'coconut_price_outcome',
 'coconut_production_outcome',
 'palm-kernel_exports_outcome',
 'palm-kernel_price_outcome',
 'palm-kernel_production_outcome',
 'palm_exports_outcome',
 'palm_price_outcome',
 'palm_production_outcome',
 'peanut_exports_outcome',
 'peanut_price_outcome',
 'peanut_production_outcome',
 'rapeseed_exports_outcome',
 'rapeseed_price_outcome',
 'rapeseed_production_outcome',
 'soybean_exports_outcome',
 'soybean_price_outcome',
 'soybean_production_outcome',
 'sunflower_exports_outcome',
 'sunflower_price_outcome',
 'sunflower_production_outcome',
 'vegetable-oil_exports_outcome',
 'vegetable-oil_price_outcome',
 'vegetable-oil_production_outcome',
 'vegetable-oil_oecd_exports_outcome',
 'vegetable-oil_oecd_production_outcome']

## Divisao em Quantis

Processo de Discretização em **n** quantis. Processo de simplificação dos dados categorizados e criacao de grupos.

Significa que para uma data qualquer... os possiveis acontecimentos predecessores *(LHM, MLH, etc)* que o fizeram chegar a aquele resultado.

In [22]:
def quantiles_v2(df: pd.DataFrame, size = 3):
    df    = df.copy()

    columns_production  = []
    columns_exports     = []
    columns_prices      = []
    columns_veg         = []
    
    for i, col in enumerate(df.columns):
        column_name         = f'{col}_qcut'
        
        # EXecuta o qcut em campos que tem no nome a referencia 'variation' mas que nao seja tipo 'vegetable'
        if 'variation' in col and 'vegetable' not in col:
        
            if 'exports' in col:
                labels = get_labels(size, 0*size)
                df[column_name]     = pd.qcut(df[col], q=size, labels = labels)
                columns_exports.append(column_name)    
                
            elif 'price' in col:
                labels = get_labels(size, 2*size)
                df[column_name]     = pd.qcut(df[col], q=size, labels = labels)
                columns_prices.append(column_name) 
                
            elif 'production' in col:
                labels = get_labels(size, 4*size)

                df[column_name]     = pd.qcut(df[col], q=size, labels = labels)
                columns_production.append(column_name)

  
        
        elif 'variation' in col and 'vegetable' in col:
            labels = get_labels(size, 6*size)
            df[column_name]     = pd.qcut(df[col], q=size, labels = labels)
            columns_veg.append(column_name) 
            
            
            
    def get_equation(columns):
        return ' + '.join( [f'df["{col}"].astype(str)' for col in columns] )
        
    df['event_pattern_production']  = eval(get_equation(columns_production)) 
    df['event_pattern_exports']     = eval(get_equation(columns_exports)) 
    df['event_pattern_prices']      = eval(get_equation(columns_prices)) 
    df['veg_pattern']               = eval(get_equation(columns_veg)) 
    total_patterns = 4
    
    outcomes = [x for x in df1.columns if 'outcome' in x  ]
    
    columns_fixed   = list(df.columns[:4])
    columns_sort    = sorted(list(df.columns[4:-total_patterns]))
#     columns_sort    = sorted(columns_production + columns_exports + columns_prices + outcomes)
    columns_event   = sorted(list(df.columns[-total_patterns:]))
    

    return df[columns_fixed + columns_event + columns_sort].sort_values(by = ['Close_Date', 'Sequence_ID'], ascending=[False, True])

    

In [23]:
df2 = quantiles_v2(df1, size=QTD_BINS)
print(df2.shape)
df2.tail()

(82688, 112)


,Sequence_ID,Close_Date,ref_date,size_dataset,event_pattern_exports,event_pattern_prices,event_pattern_production,veg_pattern,coconut_exports,coconut_exports_outcome,...,vegetable-oil_oecd_production_variation,vegetable-oil_oecd_production_variation_qcut,vegetable-oil_price,vegetable-oil_price_outcome,vegetable-oil_price_variation,vegetable-oil_price_variation_qcut,vegetable-oil_production,vegetable-oil_production_outcome,vegetable-oil_production_variation,vegetable-oil_production_variation_qcut
1,16,2002-04-30,2002-03-31,1,BBCAACC,HHHGGGG,NOOMMON,UTTUT,148.040,-0.400,...,0.004,T,47.622,1.347,-0.003,T,8591.720,32.090,0.004,T
1,1838,2002-04-30,2002-03-31,1,BBCAACC,HHHGGGG,NOOMMON,UTTUT,148.040,-0.400,...,0.004,T,47.622,1.347,-0.003,T,8591.720,32.090,0.004,T
1,7405,2002-04-30,2002-03-31,1,BBCAACC,HHHGGGG,NOOMMON,UTTUT,148.040,-0.400,...,0.004,T,47.622,1.347,-0.003,T,8591.720,32.090,0.004,T
1,7518,2002-04-30,2002-03-31,1,BBCAACC,HHHGGGG,NOOMMON,UTTUT,148.040,-0.400,...,0.004,T,47.622,1.347,-0.003,T,8591.720,32.090,0.004,T
1,9063,2002-04-30,2002-03-31,1,BBCAACC,HHHGGGG,NOOMMON,UTTUT,148.040,-0.400,...,0.004,T,47.622,1.347,-0.003,T,8591.720,32.090,0.004,T


In [24]:
# df2[PREDITOR]

## Compressing*


Unifica os *patterns* em uma coluna. 

A saida tem um shape que retorna ao tamanho tamanho do **RANDOM_SIZE**.

NAO é mais necessario a referencia de Date. O interesse eh somente pelos padroes.

Verificar:
- Se deve ou nao usar o SET. O importante é identificar as transições de estados anteriores?

In [25]:
def compress_v1(df: pd.DataFrame, **kwargs):
    """
    Nao eh aplicado o set. Consegue-se observar as transicoes de estados que anteciparam o estado corrente. 
    Pode ter repeticao. Acho mais importante! 
    """
    columns         = read_kwarg_list(kwargs,'columns')
    outcomes        = read_kwarg_list(kwargs,'outcomes')
    predictors      = read_kwarg_list(kwargs,'predictors')
    event_patterns  = [x for x in df.columns if 'event_pattern' in x  ]
    
    #Agrupa os event_pattern unicos em uma unica coluna
    if columns and len(event_patterns) >= 1:
        # Nao podia usar o set... pois precisava da repeticao para compararcao entre fors
        # da funcao build_transition_grid_v2
        df_step1 = df.groupby(columns)[event_patterns].\
                    agg(lambda x: ','.join( list( x ) ) )

    #Agrupa os veg_pattern somente em uma unica coluna
    veg_pattern = [x for x in df.columns if 'veg_pattern' in x  ]   
    if columns and len(veg_pattern) >= 1:
        df_step1a = df.groupby(columns)[veg_pattern].\
                    agg(lambda x: ','.join( list(  x ) ) )
        
        df_step1 = pd.merge(df_step1, df_step1a, on=columns, how='inner')
        

    if predictors and len(predictors) >= 1:
        df_step1b = df.groupby(columns)[predictors].mean()
        
        df_step1 = pd.merge(df_step1, df_step1b, on=columns, how='inner')        
        

    #Agrupa os valores dos outcomes pela media
    if columns and outcomes:
        df_step2 = df.groupby(columns)[outcomes].mean()
        
    compressed_set = pd.merge(df_step1, df_step2, on=columns, how='inner')
    
    return compressed_set
    
    agrupador      = 'events_pattern'
    
#     compressed_set[agrupador] = compressed_set[event_patterns].agg(','.join, axis=1)
       
    # Usando funcao de conjuntos para escolher as colunas da saida. A lista delta nao contem os agregadores tipo event_patterns
    array1  = np.array(list(compressed_set.columns))
    array2  = np.array([agrupador]) # np.array(event_patterns + [agrupador])
    delta   = np.setdiff1d(array1, array2, assume_unique=True)

    return compressed_set[ [event_patterns] + list(delta)]




## Compressing

In [26]:
df3 = compress_v1(df2, columns = ['Sequence_ID', 'Close_Date'],  outcomes = outcomes)
print(df3.shape)
df3.tail()

(9754, 30)


,,event_pattern_exports,event_pattern_prices,event_pattern_production,veg_pattern,coconut_exports_outcome,coconut_price_outcome,coconut_production_outcome,palm-kernel_exports_outcome,palm-kernel_price_outcome,palm-kernel_production_outcome,...,soybean_price_outcome,soybean_production_outcome,sunflower_exports_outcome,sunflower_price_outcome,sunflower_production_outcome,vegetable-oil_exports_outcome,vegetable-oil_price_outcome,vegetable-oil_production_outcome,vegetable-oil_oecd_exports_outcome,vegetable-oil_oecd_production_outcome
Sequence_ID,Close_Date,,,,,,,,,,,,,,,,,,,,,
9995,2017-01-31,"CBCBBAC,CBCBBAC,CBCBBAC","GGGGIIH,IIIGHIH,IIIHHIH","NOOOMNO,NOOOMNO,NOOOMNO","USUUU,UUUUU,UUUUU",0.917,115.023,0.960,0.370,135.360,4.257,...,5.197,11.983,10.940,0.000,13.750,32.857,3.448,78.253,36.130,99.260
9996,2014-02-28,"BAACABC,BAACABC,BAACABC,BAACABC,BAACABC,BAACAB...","GHGGGGG,GHHGGGG,IHHHHGH,IHHIIGH,GHGHGGG,HIHGHH...","MNNOOOO,MNNOOOO,MNNOOOO,MNNOOOO,MNNOOOO,MNNONO...","TSUTT,TTUTT,TTUTT,TTUTT,TSUTT,TSUTT,TTUTT,TUUS...",-0.028,49.568,-1.347,-1.707,41.750,2.574,...,-13.315,23.585,12.262,-26.978,14.923,21.301,-0.140,73.528,17.716,62.013
9997,2008-04-30,"ACBCCAC,ABBCCAC,ABBCCAC","IIIHIII,IIIGIII,IIIIIIH","NNOMOMO,NNONOMO,MNOMOMO","SUSTT,SUSTT,SUSTT",-3.120,52.667,-0.373,1.410,49.243,1.897,...,47.343,-16.853,7.090,43.000,11.347,1.540,5.103,30.647,18.457,33.100
9998,2006-12-31,"ACAACCB,ACAACCB,ACAACCB,ACAACCB,ACAACCB,ACAACC...","HHHGIII,HGHIHHG,HHIIHIG,IHIIHHI,HGGIGGH,HHHIHH...","MMMMMON,MMMMMON,MMMMMON,MMMMMON,MMMMMON,MMMMMO...","TTTSS,TTTSS,TUTSS,TUTSS,TSTSS,TTTSS,TUSSS",-2.173,21.238,-1.826,2.590,11.779,0.806,...,15.690,19.277,0.881,7.286,1.250,19.603,2.103,29.416,7.783,25.824
9999,2018-03-31,"BBABBAA,BBABAAA,BBABAAA,BBABAAA,BBABAAA,BBABAA...","GGGIGGG,HGGIHGG,IHHIHIH,IGGHHHH,GHGIIHH,IIHHGI...","OOONNMN,OOONNMN,OOONNMN,OOONNMN,OOONNMN,OOONNM...","SSTSU,SSTSU,SUTSU,SSTSU,STTSU,SUTSU,SUTSU,STTS...",0.532,-35.313,1.451,0.583,-17.782,3.856,...,1.812,7.441,-0.200,-2.855,3.223,5.387,-0.520,60.592,18.514,71.027


In [27]:
df3.columns

Index(['event_pattern_exports', 'event_pattern_prices',
       'event_pattern_production', 'veg_pattern', 'coconut_exports_outcome',
       'coconut_price_outcome', 'coconut_production_outcome',
       'palm-kernel_exports_outcome', 'palm-kernel_price_outcome',
       'palm-kernel_production_outcome', 'palm_exports_outcome',
       'palm_price_outcome', 'palm_production_outcome',
       'peanut_exports_outcome', 'peanut_price_outcome',
       'peanut_production_outcome', 'rapeseed_exports_outcome',
       'rapeseed_price_outcome', 'rapeseed_production_outcome',
       'soybean_exports_outcome', 'soybean_price_outcome',
       'soybean_production_outcome', 'sunflower_exports_outcome',
       'sunflower_price_outcome', 'sunflower_production_outcome',
       'vegetable-oil_exports_outcome', 'vegetable-oil_price_outcome',
       'vegetable-oil_production_outcome',
       'vegetable-oil_oecd_exports_outcome',
       'vegetable-oil_oecd_production_outcome'],
      dtype='object')

## Simplificando o Dataset


O objetivo aqui é remover variacoes de pouca relevancia.

Exemplo: 
- A flutuação de + ou - 10 centavos de dolar pode não ser representativo em um dataset de preço da tonela oleo de soja ... que opera na casa de milhares de dolares. Mas 10 centavos é representativo para o cambio dolar/real, que opera na casa  5 BRL ~ 1 USD.
- Pode ser até erro de precisão. 1000 kg ou 999 kg é a mesma coisa em termos de ordem de grandeza

In [28]:
df3[[x for x in df3.columns if 'vegetable' in x  ]].describe()

,vegetable-oil_exports_outcome,vegetable-oil_price_outcome,vegetable-oil_production_outcome,vegetable-oil_oecd_exports_outcome,vegetable-oil_oecd_production_outcome
count,9754.000,9754.000,9754.000,9754.000,9754.000
mean,19.425,0.402,45.453,19.311,45.110
std,18.467,3.645,21.655,14.660,24.796
min,-28.240,-29.180,-9.230,-24.600,-17.340
25%,8.844,-1.305,32.188,11.650,31.120
50%,20.403,0.114,47.991,22.113,43.609
75%,30.541,1.829,61.008,28.521,61.942
max,136.430,15.540,176.590,127.930,158.470


In [29]:
relevant_cut_off  = .5 #variacao maior que .5 pontos 
outcome_reference = 'vegetable-oil_price_outcome'

print(f'All available data: { df3.shape[0]}')
df4 = df3[ abs(df3[outcome_reference]) > relevant_cut_off ][:]
print(f'Relevant changes: { df4.shape[0]}')

All available data: 9754
Relevant changes: 7852


In [30]:
df4[[x for x in df4.columns if 'vegetable' in x]].describe()

,vegetable-oil_exports_outcome,vegetable-oil_price_outcome,vegetable-oil_production_outcome,vegetable-oil_oecd_exports_outcome,vegetable-oil_oecd_production_outcome
count,7852.000,7852.000,7852.000,7852.000,7852.000
mean,18.782,0.500,43.702,19.187,43.645
std,19.118,4.055,21.757,14.830,24.349
min,-28.240,-29.180,-9.230,-24.600,-17.340
25%,7.768,-1.647,30.876,11.647,31.116
50%,19.602,0.657,46.351,21.414,42.308
75%,28.597,2.395,57.432,27.927,58.311
max,136.430,15.540,176.590,127.930,158.470


## Directions

Neste ponto, o objetivo é criar um flag 0/1 (False/True) para informar se a sequencia de transição direcionar para crescimento (bulish) ou para caida (bearish).


In [31]:
def directions(df: pd.DataFrame):
    df    = df.copy()

    for col in [x for x in df.columns if 'outcome' in x ]:
        column_name         = f'{col}_direction'
        df[column_name]     = df[col].apply(lambda x: 1 if x > 0 else 0 )

    # quantidade de colunas sem outcome na descricao
    qtd_cols = len([x for x in df.columns if 'outcome' not in x])
        
    columns_fixed   = list(df.columns[:qtd_cols])
    columns_sort    = sorted(list(df.columns[qtd_cols:]))
    
#     return df[columns_fixed + columns_sort].sort_values(by = ['Sequence_ID'], ascending=[True]) \
#             .reset_index().set_index(['Sequence_ID', 'Close_Date'])

    return df[columns_fixed + columns_sort].sort_values(by = ['Close_Date'], ascending=[True]) \
            .reset_index().set_index(['Sequence_ID', 'Close_Date'])

In [32]:
#sem simplificacao
%time df5 = directions(df4)
print(df5.shape)
df5

CPU times: total: 156 ms
Wall time: 125 ms
(7852, 56)


,,event_pattern_exports,event_pattern_prices,event_pattern_production,veg_pattern,coconut_exports_outcome,coconut_exports_outcome_direction,coconut_price_outcome,coconut_price_outcome_direction,coconut_production_outcome,coconut_production_outcome_direction,...,vegetable-oil_exports_outcome,vegetable-oil_exports_outcome_direction,vegetable-oil_oecd_exports_outcome,vegetable-oil_oecd_exports_outcome_direction,vegetable-oil_oecd_production_outcome,vegetable-oil_oecd_production_outcome_direction,vegetable-oil_price_outcome,vegetable-oil_price_outcome_direction,vegetable-oil_production_outcome,vegetable-oil_production_outcome_direction
Sequence_ID,Close_Date,,,,,,,,,,,,,,,,,,,,,
7518,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,UTTUT,-0.400,0,45.000,1,-0.110,0,...,27.640,1,27.980,1,29.040,1,1.347,1,32.090,1
7405,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,UTTUT,-0.400,0,45.000,1,-0.110,0,...,27.640,1,27.980,1,29.040,1,1.347,1,32.090,1
9063,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,UTTUT,-0.400,0,45.000,1,-0.110,0,...,27.640,1,27.980,1,29.040,1,1.347,1,32.090,1
16,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,UTTUT,-0.400,0,45.000,1,-0.110,0,...,27.640,1,27.980,1,29.040,1,1.347,1,32.090,1
1838,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,UTTUT,-0.400,0,45.000,1,-0.110,0,...,27.640,1,27.980,1,29.040,1,1.347,1,32.090,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5334,2021-12-31,"AAACBBA,AAACBBA","IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM","SUSSS,STSSS",-0.575,0,167.385,1,-0.140,0,...,-15.100,0,11.645,1,18.945,1,8.654,1,2.015,1
7799,2021-12-31,AAACBBA,IIIGHHH,NMMNNMM,STSSS,-0.580,0,250.590,1,-0.140,0,...,-15.110,0,11.650,1,18.940,1,15.453,1,2.010,1
5422,2021-12-31,AAACBBA,IIIGHHH,NMMNNMM,STSSS,-0.580,0,250.590,1,-0.140,0,...,-15.110,0,11.650,1,18.940,1,15.453,1,2.010,1


## Split Train/Test

O Shuffle esta desativado. Não sera realizado embaralhamento de serie temporal, para evitar o erro de prever o passado com dados do futuro.

In [33]:
event_patterns = [x for x in df5.columns if 'event_pattern' in x  ]
print(event_patterns)

feature_directions = event_patterns + [x for x in df5.columns if 'direction' in x and 'vegetable-oil' not in x ]
print(feature_directions)

predictors = [x for x in df5.columns if 'direction' in x and 'vegetable-oil' in x and 'oecd' not in x ]
predictors

['event_pattern_exports', 'event_pattern_prices', 'event_pattern_production']
['event_pattern_exports', 'event_pattern_prices', 'event_pattern_production', 'coconut_exports_outcome_direction', 'coconut_price_outcome_direction', 'coconut_production_outcome_direction', 'palm-kernel_exports_outcome_direction', 'palm-kernel_price_outcome_direction', 'palm-kernel_production_outcome_direction', 'palm_exports_outcome_direction', 'palm_price_outcome_direction', 'palm_production_outcome_direction', 'peanut_exports_outcome_direction', 'peanut_price_outcome_direction', 'peanut_production_outcome_direction', 'rapeseed_exports_outcome_direction', 'rapeseed_price_outcome_direction', 'rapeseed_production_outcome_direction', 'soybean_exports_outcome_direction', 'soybean_price_outcome_direction', 'soybean_production_outcome_direction', 'sunflower_exports_outcome_direction', 'sunflower_price_outcome_direction', 'sunflower_production_outcome_direction']


['vegetable-oil_exports_outcome_direction',
 'vegetable-oil_price_outcome_direction',
 'vegetable-oil_production_outcome_direction']

In [78]:
X = df5[feature_directions + predictors]
y = df5[predictors]


In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=False)
print(f'X_train: {X_train.shape} | X_test: {X_test.shape} | y_train: {y_train.shape} | y_test: {y_test.shape} ')

X_train: (7459, 27) | X_test: (393, 27) | y_train: (7459, 3) | y_test: (393, 3) 


In [80]:
X_train = X_train.sort_values(by=['Sequence_ID'])
X_train.sort_values(by=['Close_Date'], ascending=[False])

,,event_pattern_exports,event_pattern_prices,event_pattern_production,coconut_exports_outcome_direction,coconut_price_outcome_direction,coconut_production_outcome_direction,palm-kernel_exports_outcome_direction,palm-kernel_price_outcome_direction,palm-kernel_production_outcome_direction,palm_exports_outcome_direction,...,rapeseed_production_outcome_direction,soybean_exports_outcome_direction,soybean_price_outcome_direction,soybean_production_outcome_direction,sunflower_exports_outcome_direction,sunflower_price_outcome_direction,sunflower_production_outcome_direction,vegetable-oil_exports_outcome_direction,vegetable-oil_price_outcome_direction,vegetable-oil_production_outcome_direction
Sequence_ID,Close_Date,,,,,,,,,,,,,,,,,,,,,
985,2020-09-30,"AAACBBA,AAACBBA,AAACBBA,AAACBBA","HGGIIHH,IIIIIII,GGIIIII,IIIHIII","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
5716,2020-09-30,"AAABBCB,AAABBCB,AAABBCB,AAACBBA,AAACBBA,AAACBB...","HGHHHHH,IIIGGHH,IIIIHIH,GHIGIII,GGGIGGI,HGGIGG...","MMMOMNN,MMMOMNN,MMMOMNN,NMMNNMM,NMMNNMM,NMMNNM...",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
9168,2020-09-30,"AAABBCB,AAABBCB,AAABBCB,AAABBCB,AAABBCB,AAACBB...","IIIIIII,HHHHIGH,HGHHHHH,IIIGGHH,IIIIHIH,GHIGII...","MMMOMNN,MMMOMNN,MMMOMNN,MMMOMNN,MMMOMNN,NMMNNM...",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
917,2020-09-30,"AAACBBA,AAACBBA,AAACBBA","IIIIIII,GGIIIII,IIIHIII","NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
1947,2020-09-30,"AAABBCB,AAABBCB,AAABBCB,AAACBBA,AAACBBA,AAACBB...","HGHHHHH,IIIGGHH,IIIIHIH,GHIGIII,GGGIGGI,HGGIGG...","MMMOMNN,MMMOMNN,MMMOMNN,NMMNNMM,NMMNNMM,NMMNNM...",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,0,1,0,1,1,1,1,...,0,1,1,1,1,0,1,1,1,1
1838,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,0,1,0,1,1,1,1,...,0,1,1,1,1,0,1,1,1,1
7518,2002-04-30,BBCAACC,HHHGGGG,NOOMMON,0,1,0,1,1,1,1,...,0,1,1,1,1,0,1,1,1,1


In [81]:
X_test = X_test.sort_values(by=['Sequence_ID'])
X_test.sort_values(by=['Close_Date'], ascending=[False])

,,event_pattern_exports,event_pattern_prices,event_pattern_production,coconut_exports_outcome_direction,coconut_price_outcome_direction,coconut_production_outcome_direction,palm-kernel_exports_outcome_direction,palm-kernel_price_outcome_direction,palm-kernel_production_outcome_direction,palm_exports_outcome_direction,...,rapeseed_production_outcome_direction,soybean_exports_outcome_direction,soybean_price_outcome_direction,soybean_production_outcome_direction,sunflower_exports_outcome_direction,sunflower_price_outcome_direction,sunflower_production_outcome_direction,vegetable-oil_exports_outcome_direction,vegetable-oil_price_outcome_direction,vegetable-oil_production_outcome_direction
Sequence_ID,Close_Date,,,,,,,,,,,,,,,,,,,,,
3674,2021-12-31,"AAACBBA,AAACBBA","IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
5334,2021-12-31,"AAACBBA,AAACBBA","IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
5422,2021-12-31,AAACBBA,IIIGHHH,NMMNNMM,0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
7387,2021-12-31,AAACBBA,IIIGHHH,NMMNNMM,0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
8094,2021-12-31,"AAACBBA,AAACBBA,AAACBBA","IIIHIII,IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8219,2020-10-31,"AAABBCB,AAABBCB,AAABBCB,AAABBCB,AAACBBA,AAACBB...","HHHHIGH,HGHHHHH,IIIGGHH,IIIIHIH,GHIGIII,GGGIGG...","MMMOMNN,MMMOMNN,MMMOMNN,MMMOMNN,NMMNNMM,NMMNNM...",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
8243,2020-10-31,"AAABBCC,AAABBCC,AAABBCB,AAABBCB,AAABBCB,AAABBC...","GGGGIHH,HHHIHHI,IIIIIII,HHHHIGH,HGHHHHH,IIIGGH...","MMMONNN,MMMOMNN,MMMOMNN,MMMOMNN,MMMOMNN,MMMOMN...",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1
8258,2020-10-31,"AAABBCC,AAABBCC,AAABBCB,AAABBCB,AAABBCB,AAABBC...","GGGGIHH,HHHIHHI,IIIIIII,HHHHIGH,HGHHHHH,IIIGGH...","MMMONNN,MMMOMNN,MMMOMNN,MMMOMNN,MMMOMNN,MMMOMN...",0,1,0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,1


In [82]:
[x for x in df5.columns if 'direction' in x ]

['coconut_exports_outcome_direction',
 'coconut_price_outcome_direction',
 'coconut_production_outcome_direction',
 'palm-kernel_exports_outcome_direction',
 'palm-kernel_price_outcome_direction',
 'palm-kernel_production_outcome_direction',
 'palm_exports_outcome_direction',
 'palm_price_outcome_direction',
 'palm_production_outcome_direction',
 'peanut_exports_outcome_direction',
 'peanut_price_outcome_direction',
 'peanut_production_outcome_direction',
 'rapeseed_exports_outcome_direction',
 'rapeseed_price_outcome_direction',
 'rapeseed_production_outcome_direction',
 'soybean_exports_outcome_direction',
 'soybean_price_outcome_direction',
 'soybean_production_outcome_direction',
 'sunflower_exports_outcome_direction',
 'sunflower_price_outcome_direction',
 'sunflower_production_outcome_direction',
 'vegetable-oil_exports_outcome_direction',
 'vegetable-oil_oecd_exports_outcome_direction',
 'vegetable-oil_oecd_production_outcome_direction',
 'vegetable-oil_price_outcome_direction',

In [83]:
print(X_train.shape, "Metade de linhas:", X_train.shape[0]/2)
print('=='*60)
print('Vou usar a referencia de "sunflower_production_outcome_direction" por ser o fator determinando pertubador de mercado. ')
print('=='*60)

X_train[[x for x in X_train.columns if 'direction' in x ]].sum()

(7459, 27) Metade de linhas: 3729.5
Vou usar a referencia de "sunflower_production_outcome_direction" por ser o fator determinando pertubador de mercado. 


coconut_exports_outcome_direction             4133
coconut_price_outcome_direction               3995
coconut_production_outcome_direction          4204
palm-kernel_exports_outcome_direction         5280
palm-kernel_price_outcome_direction           3892
palm-kernel_production_outcome_direction      6593
palm_exports_outcome_direction                6362
palm_price_outcome_direction                  3754
palm_production_outcome_direction             6685
peanut_exports_outcome_direction              4335
peanut_price_outcome_direction                4202
peanut_production_outcome_direction           4628
rapeseed_exports_outcome_direction            6322
rapeseed_price_outcome_direction              3948
rapeseed_production_outcome_direction         6446
soybean_exports_outcome_direction             4283
soybean_price_outcome_direction               3873
soybean_production_outcome_direction          6519
sunflower_exports_outcome_direction           5318
sunflower_price_outcome_directi

## Train/Test 2

Neste caso não estou usando a biblioteca. Divisao simples

## Bullish/Bearish Cenario

In [84]:
direction_reference = 'sunflower_production_outcome_direction'
df6_pos = X_train.loc[X_train[direction_reference] > 0]
df6_neg = X_train.loc[X_train[direction_reference] <= 0]
print(f'df6_pos: {df6_pos.shape} | df6_neg: {df6_neg.shape}')
df6_pos.head()

df6_pos: (5755, 27) | df6_neg: (1704, 27)


,,event_pattern_exports,event_pattern_prices,event_pattern_production,coconut_exports_outcome_direction,coconut_price_outcome_direction,coconut_production_outcome_direction,palm-kernel_exports_outcome_direction,palm-kernel_price_outcome_direction,palm-kernel_production_outcome_direction,palm_exports_outcome_direction,...,rapeseed_production_outcome_direction,soybean_exports_outcome_direction,soybean_price_outcome_direction,soybean_production_outcome_direction,sunflower_exports_outcome_direction,sunflower_price_outcome_direction,sunflower_production_outcome_direction,vegetable-oil_exports_outcome_direction,vegetable-oil_price_outcome_direction,vegetable-oil_production_outcome_direction
Sequence_ID,Close_Date,,,,,,,,,,,,,,,,,,,,,
0,2003-06-30,"BCCCCBB,BCCCCBB,BCCCCBB","GGGIGHG,GGGIHII,IHIIIHI","NOOOOMO,NOOOOMO,NOOOOMO",1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
1,2012-08-31,"BABBCAC,BABBCAC,BABBCAC,BABBCAC,BABBCAC,BABBCA...","HIHHGHH,GGGIHHI,GGHIHHG,GHHIGHH,GGGIGGG,GGGHGG...","MNNNNMO,MNNNNMO,MNNNNMO,MNNNNMO,MNNNNMO,MNNNNM...",1,0,1,1,0,1,1,...,1,0,0,1,1,0,1,1,0,1
4,2004-10-31,"BCCCCBB,CCCAACA,CCCAACA,CCCAACA,CCCAACA,CCCAAC...","IIHHIHI,HHGIIII,IIIGIII,IIIGGHG,IIHGGGG,GHGGIG...","NOOOOMN,OOOMOOM,OOOMOOM,OOOMOOM,OOOMOOM,OOOMOO...",1,1,1,1,1,1,1,...,1,1,0,1,0,1,1,1,0,1
5,2003-06-30,"ABCAACC,ABCAACC,ABCAACC,ACCAACC,BCCCCBB,BCCCCB...","HHIIHII,IIIIIII,HHIIIII,IIIIIIH,GGHIGGG,GGGIGH...","NOOMMON,NOOMMON,NOOMMOO,NOOMMOO,NOOOOMO,NOOOOM...",0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,2010-08-31,"CBBBBCB,CBBBBCB,CBBBBCA,CBBBBCB,CBBBBCB,CBBBBC...","GGGGGGG,IIIGIII,GGGGGGH,HHIGIII,HHIGIII,IIIIHH...","ONMMOON,ONMMOON,ONMMOON,ONMMOON,ONMMOON,ONMMOO...",1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1


## Unique Patterns

In [41]:
def get_unique_patterns_V1(input_array: np.ndarray, **kwargs):
    """ 
    Aqui o 'get_labels' nao deve fazer parte... pois os unique patterns pode estar agrupado. Ex. AAA, ABC, etc
    """
    flat_list = [ item.split(',') for item in input_array ]
    unique_patterns = ','.join(str(r) for v in flat_list for r in v)
    unique_patterns = sorted(list( set( unique_patterns.split(',') ) ))
    return unique_patterns

def get_unique_patterns_V2(input_array: np.ndarray, **kwargs):
    """ 
    Aqui o 'get_labels' nao deve fazer parte... pois os unique patterns pode estar agrupado. Ex. AAA, ABC, etc
    """
    unique_patterns = []
    for i in range(len(input_array[0])):

        flat_list = [ item[i].split(',') for item in input_array ]
        temp_patterns = ','.join(str(r) for v in flat_list for r in v)
        unique_patterns += temp_patterns.split(',')
                
    unique_patterns = sorted(list( set(unique_patterns ) ))
    return unique_patterns


def get_unique_patterns_V3(input_array: np.ndarray, **kwargs):
    """ 
    Aqui o 'get_labels' nao deve fazer parte... pois os unique patterns pode estar agrupado. Ex. AAA, ABC, etc
    """
    unique_patterns = {}
    for i in range(len(input_array[0])):

        flat_list = [ item[i].split(',') for item in input_array ]
        temp_patterns = ','.join(str(r) for v in flat_list for r in v)
        unique_patterns[i] += temp_patterns.split(',')
                
#     unique_patterns = sorted(list( set(unique_patterns ) ))
    return unique_patterns

In [42]:
## X_train[event_patterns].values[1][0].split(",")[0].split('-')

In [43]:
# flat_list = [ item[0].split(',') for item in X_train[event_patterns].values ]
# flat_list = [ j[0].split('-') for j in   [i[0].split(",") for i in X_train[event_patterns].values]] 
# flat_list
# # unique_patterns = ','.join(str(r) for v in flat_list for r in v)
# # unique_patterns = sorted(list( set( unique_patterns.split(',') ) ))
# unique_patterns

In [44]:
unique_patterns_exports = get_unique_patterns_V1(X_train['event_pattern_exports'].values)
print(len(unique_patterns_exports), unique_patterns_exports[:])

32 ['AAAAAAB', 'AAAACBB', 'AAABBCB', 'AAABBCC', 'AAACBBA', 'AABACBB', 'AABBCBB', 'ABBCCAC', 'ABCAACC', 'ACAACCB', 'ACBCCAC', 'ACCAACC', 'BAACABC', 'BABBCAC', 'BBABAAA', 'BBABBAA', 'BBBCCBC', 'BBCAACC', 'BBCCCBC', 'BCBAACA', 'BCCCBCA', 'BCCCCBB', 'CABBCAC', 'CBBBBCA', 'CBBBBCB', 'CBCAAAA', 'CBCBBAC', 'CBCCBAC', 'CCBAACA', 'CCBBBBB', 'CCCAABA', 'CCCAACA']


In [45]:
unique_patterns_prices = get_unique_patterns_V1(X_train['event_pattern_prices'].values)
print(len(unique_patterns_prices), unique_patterns_prices[:])

155 ['GGGGGGG', 'GGGGGGH', 'GGGGGHG', 'GGGGHGG', 'GGGGHGH', 'GGGGIHG', 'GGGGIHH', 'GGGGIIH', 'GGGHGGG', 'GGGHGGH', 'GGGHGGI', 'GGGHHGG', 'GGGHHHH', 'GGGHHHI', 'GGGIGGG', 'GGGIGGI', 'GGGIGHG', 'GGGIHHH', 'GGGIHHI', 'GGGIHII', 'GGGIIHG', 'GGHGGGH', 'GGHGGHG', 'GGHGHGG', 'GGHGHGH', 'GGHGHHH', 'GGHGIHI', 'GGHHGGG', 'GGHHGHH', 'GGHIGGG', 'GGHIHHG', 'GGIGGHH', 'GGIGGII', 'GGIIIII', 'GHGGGGG', 'GHGGIGG', 'GHGGIHI', 'GHGHGGG', 'GHGIIHH', 'GHHGGGG', 'GHHGIHH', 'GHHGIHI', 'GHHGIII', 'GHHIGHH', 'GHIGIIG', 'GHIGIII', 'GHIHHIH', 'GIHGIHG', 'GIIGHIH', 'HGGGGGG', 'HGGGHHG', 'HGGGHHI', 'HGGIGGG', 'HGGIGGH', 'HGGIGHG', 'HGGIHGG', 'HGGIHHH', 'HGGIIHH', 'HGHGGHG', 'HGHHHHH', 'HGHIHHG', 'HGHIIHH', 'HGIHHGH', 'HHGGGGG', 'HHGGGIH', 'HHGGHGH', 'HHGGHGI', 'HHGGHHH', 'HHGHGGG', 'HHGHHHH', 'HHGIGGH', 'HHGIIII', 'HHHGGGG', 'HHHGGGH', 'HHHGGHG', 'HHHGGHI', 'HHHGGIH', 'HHHGHGG', 'HHHGHHG', 'HHHGHHH', 'HHHGIHI', 'HHHGIII', 'HHHHGGG', 'HHHHHGI', 'HHHHHHG', 'HHHHHHH', 'HHHHIGH', 'HHHHIHH', 'HHHIHHG', 'HHHIHHH', 'HHHI

In [46]:
unique_patterns_production = get_unique_patterns_V1(X_train['event_pattern_production'].values)
print(len(unique_patterns_production), unique_patterns_production[:])

37 ['MMMMMON', 'MMMNMNN', 'MMMOMNN', 'MMMONNN', 'MNMONNO', 'MNNMOMO', 'MNNNNMO', 'MNNNOOO', 'MNNONOO', 'MNNOOOO', 'MNOMOMO', 'MONNNMO', 'NMMMNOM', 'NMMNMNN', 'NMMNNMM', 'NMNMMNN', 'NMNOMNM', 'NNNMMNN', 'NNNMOMO', 'NNOMOMO', 'NNONOMO', 'NOOMMON', 'NOOMMOO', 'NOOOMNO', 'NOOOOMN', 'NOOOOMO', 'OMNOMNM', 'ONMMOON', 'ONNNNMM', 'OONNNMM', 'OOOMOOM', 'OOONNMM', 'OOONNMN', 'OOONNNM', 'OOOONMM', 'OOOONMN', 'OOOOOMM']


## Matriz de Markov

Matriz estocastica
- cada elemento da matriz é positivo
- a somatoria de cada linha é 1

In [47]:
def build_grid_markov(patterns, counts, counts_fk):

#     log.debug(f'patterns: {patterns}')
#     log.debug(f'counts: {counts}')
#     log.debug(f'counts_fk: {counts_fk}')

    # create to/from grid
    grid_markov = pd.DataFrame({'pairs':patterns, 'counts': counts})
    log.debug(f'CRIACAO GRID: {grid_markov.shape} | {grid_markov}')
    
    # group by, para remover as duplicacoes de multiplos patterns
    grid_markov = grid_markov.groupby(['pairs'])['counts'].sum().to_frame().reset_index()
    log.debug(f'GRID GROUPED: {grid_markov.shape} | {grid_markov}')

    # quebra em x,y a coluna combinada
    grid_markov[['x', 'y']] = grid_markov['pairs'].str.split(',', n=1, expand=True)
    log.debug(f'GRID X,Y: {grid_markov.shape} | {grid_markov}')

    # pivoteamento em x e y
    grid_markov = grid_markov.pivot(index='x', columns='y', values='counts')
    log.debug(f'GRID PIVOT: {grid_markov.shape} | {grid_markov}')
    
    # Renomeia as colunas. Remove a referencia 'y'
    grid_markov.columns= [col for col in grid_markov.columns]
    log.debug(f'GRID RENAME COLUMNS: {grid_markov}')
       
    # replace all NaN with zeros
    grid_markov.fillna(0, inplace=True)
    log.debug(f'GRID FILLNA: {grid_markov}')
    
    # cria uma coluna temporaria para a soma da linha
    grid_markov['soma'] = grid_markov.sum(axis=1)
    log.debug(f'GRID SOMA: {grid_markov.shape} | {grid_markov}')

    # grid_markov.rowSums(transition_dataframe) 
    # grid_markov = grid_markov / grid_markov['soma']
    
    # calcula o percentual de cada valor sobre a soma    
    for col in grid_markov.columns:
        grid_markov[col] = grid_markov[col] / grid_markov['soma'] 
        grid_markov.fillna(0, inplace=True)
    log.debug(f'GRID PERCENT: {grid_markov.shape} | {grid_markov}')
    
    ## Correcao onde soma eh zero
    # for i, row in grid_markov[grid_markov['soma'] == 0].iterowss

    
        
    # Para o caso da divisao por soma zero. Sera criado um proporcional 1/total de colunas.
    # Assim garantindo que a soma da linha da 1.
    # O Total de colunas tem que ser respectivo de onde correu a combinacao. Ex. event_pattern_prices
#     grid_markov.fillna(1/(grid_markov.shape[1] - 1), inplace=True)
#     log.debug(f'GRID FILLNA :{grid_markov}')
#     for idx in x[x[x.columns[0]].isna()].index:
#         print(idx)
    
    #Remove a coluna Soma    
    del grid_markov['soma']
    
    log.debug(f'Prova dos Nove - Somatorio deve ser 1 | {grid_markov.T.sum()}')

    return grid_markov

In [48]:
def build_transition_grid_v1(df: pd.DataFrame, unique_patterns, pattern_column):
    '''
    build the markov transition grid
    '''
    patterns  = []
    counts    = []
    counts_fk = {}
    stop      = 0
    
    event_patterns  = [x for x in df.columns if pattern_column in x  ]

    #para cada pattern
    for col in event_patterns:

        # de
        for from_event in unique_patterns:
            
            # para
            for to_event in unique_patterns:
                pattern = f'{from_event},{to_event}' # MMM,MlM

                # captura a transicao de um estado para outro
                ids_matches = df[df[col].str.contains(pattern)]
                #print(ids_matches)

                found = 0
                if len(ids_matches) > 0:
                    Event_Pattern = '---'.join(ids_matches[col].values)
                    found = Event_Pattern.count(pattern)
                    log.debug(f'pattern => {pattern} | ids_matches: {len(ids_matches)} | found: {found} | {stop}')
                    
                patterns.append(pattern)
                counts.append(found)

                counts_fk[pattern] = f'{len(ids_matches)}|{found}'

            
#             stop +=1
#             if stop>5:
#                 break
#                 None
#     logging.basicConfig(level=logging.DEBUG)    
    return build_grid_markov(patterns, counts, counts_fk)
    

reset_logging()
    
# logging.basicConfig(level=logging.DEBUG)
# x = build_transition_grid_v1(df6_neg, unique_patterns[194:])
# x
# # x.T.sum()

## Classes + Def Matriz Markov

In [49]:
CLASSES       = ['production', 'exports', 'prices']

def get_markov(df6_pos, df6_neg):

    grids_markov = {}

    for ref in CLASSES:
        print(ref)
        pattern_column = f'event_pattern_{ref}'
        unique_patterns = get_unique_patterns_V1(X_train[pattern_column].values)

        grid_pos = build_transition_grid_v1(df6_pos, unique_patterns, pattern_column) 
        grids_markov[f'{ref}_positive'] = grid_pos
        grid_pos.to_excel(f'grid_alt3_{ref}_positive.xlsx')
        print(ref, 'positive')

        grid_neg = build_transition_grid_v1(df6_neg, unique_patterns, pattern_column)
        grids_markov[f'{ref}_negative'] = grid_neg
        grid_neg.to_excel(f'grid_alt3_{ref}_negative.xlsx') 
        print(ref, 'negative')
        
    return grids_markov


In [50]:
%time grids_markov = get_markov(df6_pos, df6_neg)

production
production positive
production negative
exports
exports positive
exports negative
prices
prices positive
prices negative
CPU times: total: 2min 7s
Wall time: 2min 7s


## Pickle

In [51]:
# grids_markov

In [52]:
import pickle

pickle.dump(grids_markov, open("grid_markov.pickle", "wb")) 

In [56]:
temp = pickle.load(open("grid_markov.pickle", "rb"))
temp

{'production_positive':          MMMMMON  MMMNMNN  MMMOMNN  MMMONNN  MNMONNO  MNNMOMO  MNNNNMO  \
 x                                                                        
 MMMMMON    0.963    0.000    0.000    0.000    0.000    0.000    0.000   
 MMMNMNN    0.000    0.000    0.000    0.000    0.000    0.000    0.000   
 MMMOMNN    0.000    0.000    0.393    0.557    0.000    0.000    0.000   
 MMMONNN    0.000    0.000    1.000    0.000    0.000    0.000    0.000   
 MNMONNO    0.000    0.000    0.000    0.000    0.613    0.000    0.000   
 MNNMOMO    0.000    0.000    0.000    0.000    0.000    0.000    0.000   
 MNNNNMO    0.000    0.000    0.000    0.000    0.000    0.000    0.910   
 MNNNOOO    0.263    0.000    0.000    0.000    0.000    0.000    0.000   
 MNNONOO    0.000    0.000    0.000    0.000    0.490    0.000    0.000   
 MNNOOOO    0.000    0.000    0.000    0.000    0.000    0.000    0.000   
 MNOMOMO    0.000    0.000    0.000    0.000    0.000    0.000    0.000   
 M

# Matriz Transição
 Markov State Transition model

- https://networks.quantecon.org/ch_mcs.html
- https://python-advanced.quantecon.org/stationary_densities.html
- https://python.quantecon.org/finite_markov.html
- https://python.quantecon.org/markov_perf.html
- https://www.viralml.com/video-content.html?fm=yt&v=sdp49vTanSk
- https://pkghosh.wordpress.com/2015/07/06/customer-conversion-prediction-with-markov-chain-classifier/
- https://rubikscode.net/2021/09/06/stock-price-prediction-using-hidden-markov-model/

$logOdds = Σlog( tp(i,j) \div tn(i,j) )$

onde
- tp(i,j) = probabilidade de transição para transição do estado i para j para valor de classe positivo (T)
- tn(i,j) = transição probabilidade de transição do estado i para j para valor de classe negativo (F)

In [54]:
def safe_log(numerator, denominator):

    if numerator <= 0 and denominator <= 0:
        log_value = 0
    elif denominator <= 0:
        log_value = np.log(numerator / 0.00001)
    elif numerator <= 0:
        log_value = np.log(0.00001 / denominator)
    else:
        log_value = np.log(numerator / denominator)
        
    return log_value

In [91]:
def predict_something(df_validation: pd.DataFrame, 
                      df_positive: pd.DataFrame, 
                      df_negative: pd.DataFrame,
                      pattern_name):
    
    actual = []
    predicted = []
    
    #Lista de indices
    list_positive = list(df_positive)
    list_negative = list(df_negative)
    total_size = len(df_validation)
    
    for row_num, seq_id in enumerate(df_validation['Sequence_ID'].values):
        patterns = df_validation[df_validation['Sequence_ID'] == seq_id][pattern_name].values[0].split(',')
        
        log.debug(f'patterns: {len(patterns)} | {patterns} | {seq_id}' )
        
        pos = []
        neg = []
        log_odds = []

        for id in range(0, len(patterns) - 1):
            
#             log.debug(f'compare 1: {patterns[id]} in { list_positive}')
#             log.debug(f'compare 2: {patterns[id+1]} in { list_positive}')
#             log.debug(f'compare 3: {patterns[id]} in { list_negative}')
#             log.debug(f'compare 4: {patterns[id+1]} in { list_negative}')


            # get log odds
            # logOdds = log(tp(i,j) / tn(i,j)
            
            if patterns[id] in list_positive and patterns[id+1] in list_positive and \
                patterns[id] in list_negative and patterns[id+1] in list_negative:

                numerator = df_positive[patterns[id]][patterns[id+1]]
                denominator = df_negative[patterns[id]][patterns[id+1]]
                
                log_value = safe_log(numerator, denominator)
                
#                 log.debug(f'numerator: {numerator:.5f} | denominator: {denominator:.5f} | log_value : {log_value:.5f}')

            else:
                continue
                log_value = 0

            log_odds.append(log_value)
            pos.append(numerator)
            neg.append(denominator)
            
#             break

        summarized_log = sum(log_odds)
    
#         log.debug(f'log_value:{sum(log_odds)}')
#         log.debug(f'numerator:{sum(pos)}')
#         log.debug(f'denominator:{sum(neg)}')
            
        print('===' * 30)
        outcome = df_validation[df_validation["Sequence_ID"]==seq_id]["vegetable-oil_price_outcome_direction"].values[0]
        
        model_prediction = 1 if summarized_log > 0 else -1
        
        def msg():
            print(f'row_num: {row_num+1} | seq_id: {seq_id} | Processed : {((row_num+1)/total_size)*100:.2f}%')
            print(f'sum(pos): {sum(pos)} | sum(neg): {sum(neg)}')# | sum(pos)/sum(neg) : {sum(pos)/sum(neg)} ')
            print(f'sum(log_odds): {summarized_log}')
            print(f'Outcome: {outcome} | Predict: {model_prediction}')
        
        if model_prediction == outcome:
            print('Model Result vs. Actual is **CORRECT**')
            msg()
        else: 
            print('Model Result vs. Actual is **INCORRECT**')
            

        actual.append(outcome)
        predicted.append(model_prediction)
        
#         break
        
    return actual, predicted


reset_logging()

logging.basicConfig(level=logging.DEBUG)

actual, predicted = predict_something(X_test.reset_index(), grid_pos, grid_neg, "event_pattern_production")

DEBUG:__main__:patterns: 4 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3
DEBUG:__main__:patterns: 10 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 6
DEBUG:__main__:patterns: 4 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 44
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 58
DEBUG:__main__:patterns: 1 | ['NMMNNMM'] | 65
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 66
DEBUG:__main__:patterns: 3 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 69
DEBUG:__main__:patterns: 10 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 93
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 111
DEBUG:__main__:patterns: 9 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 152
DEBUG:__main__:patterns: 4 | ['N

Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs.

DEBUG:__main__:patterns: 8 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1792
DEBUG:__main__:patterns: 4 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1796
DEBUG:__main__:patterns: 10 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1802
DEBUG:__main__:patterns: 7 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1839
DEBUG:__main__:patterns: 12 | ['MMMOMNN', 'MMMOMNN', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1847
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1906
DEBUG:__main__:patterns: 6 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1920
DEBUG:__main__:patterns: 7 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 1922
DEBUG:__main__:patterns: 7 | ['NMMNNMM', 'NMMNNMM', '

Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs.

DEBUG:__main__:patterns: 2 | ['NMMNNMM', 'NMMNNMM'] | 3674
DEBUG:__main__:patterns: 1 | ['NMMNNMM'] | 3700
DEBUG:__main__:patterns: 11 | ['MMMOMNN', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3723
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3813
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3854
DEBUG:__main__:patterns: 6 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3855
DEBUG:__main__:patterns: 4 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3857
DEBUG:__main__:patterns: 3 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3859
DEBUG:__main__:patterns: 11 | ['MMMOMNN', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 3864
DEBUG:__main__:patterns: 2 | ['NMMNNMM', 'NMMNNMM'] | 3875
DEBUG:__main__:patterns: 9 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMM

...

DEBUG:__main__:patterns: 4 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5691
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5723
DEBUG:__main__:patterns: 7 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5746
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5757
DEBUG:__main__:patterns: 9 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5790
DEBUG:__main__:patterns: 6 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5826
DEBUG:__main__:patterns: 1 | ['NMMNNMM'] | 5831
DEBUG:__main__:patterns: 6 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5840
DEBUG:__main__:patterns: 8 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5882
DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 5960
DEBUG:__main__

DEBUG:__main__:patterns: 5 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 7915
DEBUG:__main__:patterns: 6 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 7966
DEBUG:__main__:patterns: 2 | ['NMMNNMM', 'NMMNNMM'] | 7991
DEBUG:__main__:patterns: 4 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8003
DEBUG:__main__:patterns: 10 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8025


Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs.

DEBUG:__main__:patterns: 15 | ['MMMOMNN', 'MMMOMNN', 'MMMOMNN', 'MMMOMNN', 'MMMOMNN', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8055
DEBUG:__main__:patterns: 3 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8094
DEBUG:__main__:patterns: 4 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8145
DEBUG:__main__:patterns: 8 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8151
DEBUG:__main__:patterns: 2 | ['NMMNNMM', 'NMMNNMM'] | 8183
DEBUG:__main__:patterns: 3 | ['NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8212
DEBUG:__main__:patterns: 2 | ['NMMNNMM', 'NMMNNMM'] | 8217
DEBUG:__main__:patterns: 2 | ['NMMNNMM', 'NMMNNMM'] | 8218
DEBUG:__main__:patterns: 13 | ['MMMOMNN', 'MMMOMNN', 'MMMOMNN', 'MMMOMNN', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM', 'NMMNNMM'] | 8219
DEBUG:__main__:patterns: 16 | ['MMMONNN', 'MMMOMNN', 'MMMOMNN', 'MMMOMNN', 'MMMOMNN', 'MMMOMN

Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs. Actual is **INCORRECT**
Model Result vs.

In [77]:
list(X_test.reset_index().columns)

['Sequence_ID',
 'Close_Date',
 'event_pattern_exports',
 'event_pattern_prices',
 'event_pattern_production',
 'coconut_exports_outcome_direction',
 'coconut_price_outcome_direction',
 'coconut_production_outcome_direction',
 'palm-kernel_exports_outcome_direction',
 'palm-kernel_price_outcome_direction',
 'palm-kernel_production_outcome_direction',
 'palm_exports_outcome_direction',
 'palm_price_outcome_direction',
 'palm_production_outcome_direction',
 'peanut_exports_outcome_direction',
 'peanut_price_outcome_direction',
 'peanut_production_outcome_direction',
 'rapeseed_exports_outcome_direction',
 'rapeseed_price_outcome_direction',
 'rapeseed_production_outcome_direction',
 'soybean_exports_outcome_direction',
 'soybean_price_outcome_direction',
 'soybean_production_outcome_direction',
 'sunflower_exports_outcome_direction',
 'sunflower_price_outcome_direction',
 'sunflower_production_outcome_direction']

In [62]:
print(grids_markov['production_positive'].shape)
print(grids_markov['production_negative'].shape)

grid_pos = grids_markov['production_positive']
grid_neg = grids_markov['production_negative']

(37, 37)
(37, 37)


In [75]:
grid_pos

,MMMMMON,MMMNMNN,MMMOMNN,MMMONNN,MNMONNO,MNNMOMO,MNNNNMO,MNNNOOO,MNNONOO,MNNOOOO,...,ONMMOON,ONNNNMM,OONNNMM,OOOMOOM,OOONNMM,OOONNMN,OOONNNM,OOOONMM,OOOONMN,OOOOOMM
x,,,,,,,,,,,,,,,,,,,,,
MMMMMON,0.963,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.037
MMMNMNN,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
MMMOMNN,0.000,0.000,0.393,0.557,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
MMMONNN,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
MNMONNO,0.000,0.000,0.000,0.000,0.613,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
MNNMOMO,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
MNNNNMO,0.000,0.000,0.000,0.000,0.000,0.000,0.910,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.090,0.000,0.000,0.000
MNNNOOO,0.263,0.000,0.000,0.000,0.000,0.000,0.000,0.737,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
MNNONOO,0.000,0.000,0.000,0.000,0.490,0.000,0.000,0.000,0.510,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [67]:
list(X_test.reset_index().columns)
X_test.reset_index()

,Sequence_ID,Close_Date,event_pattern_exports,event_pattern_prices,event_pattern_production,coconut_exports_outcome_direction,coconut_price_outcome_direction,coconut_production_outcome_direction,palm-kernel_exports_outcome_direction,palm-kernel_price_outcome_direction,...,peanut_production_outcome_direction,rapeseed_exports_outcome_direction,rapeseed_price_outcome_direction,rapeseed_production_outcome_direction,soybean_exports_outcome_direction,soybean_price_outcome_direction,soybean_production_outcome_direction,sunflower_exports_outcome_direction,sunflower_price_outcome_direction,sunflower_production_outcome_direction
0,3,2020-10-31,"AAACBBA,AAACBBA,AAACBBA,AAACBBA","IIIIIII,GGIIIII,IIIHIII,IHIHHII","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
1,6,2021-03-31,"AAACBBA,AAACBBA,AAACBBA,AAACBBA,AAACBBA,AAACBB...","GHIGIII,GGGIGGI,HGGIGGG,HIGIGGH,HGGIIHH,IIIIII...","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM,NMMNNM...",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
2,44,2020-12-31,"AAACBBA,AAACBBA,AAACBBA,AAACBBA","GGIIIII,IIIHIII,IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
3,58,2021-04-30,"AAACBBA,AAACBBA,AAACBBA,AAACBBA,AAACBBA","IIIIIII,GGIIIII,IIIHIII,IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
4,65,2021-08-31,AAACBBA,IIIGHHH,NMMNNMM,0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,9935,2020-11-30,"AAACBBA,AAACBBA,AAACBBA,AAACBBA","GGIIIII,IIIHIII,IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
389,9941,2021-04-30,"AAACBBA,AAACBBA,AAACBBA,AAACBBA","GGIIIII,IIIHIII,IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
390,9946,2021-01-31,"AAABBCB,AAABBCB,AAABBCB,AAABBCB,AAACBBA,AAACBB...","HHHHIGH,HGHHHHH,IIIGGHH,IIIIHIH,GHIGIII,GGGIGG...","MMMOMNN,MMMOMNN,MMMOMNN,MMMOMNN,NMMNNMM,NMMNNM...",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0
391,9962,2020-12-31,"AAACBBA,AAACBBA,AAACBBA,AAACBBA","GGIIIII,IIIHIII,IHIHHII,IIIGHHH","NMMNNMM,NMMNNMM,NMMNNMM,NMMNNMM",0,1,0,0,1,...,1,1,1,1,1,1,1,0,1,0


## AA

In [92]:
P = [[0.9, 0.1, 0.0],
     [0.4, 0.4, 0.2],
     [0.1, 0.1, 0.8]]
mc = qe.MarkovChain(P, ('poor', 'middle', 'rich'))
mc.is_irreducible

True

In [95]:
mc

Markov chain with transition matrix 
P = 
[[0.9 0.1 0. ]
 [0.4 0.4 0.2]
 [0.1 0.1 0.8]]
and stationary distributions 
[[0.71428571 0.14285714 0.14285714]]

In [93]:
mc.period

1

In [94]:
mc = qe.MarkovChain(P)
mc.stationary_distributions 

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=109)
           2	LOAD_FAST(arg=0, lineno=109)
           4	LOAD_ATTR(arg=0, lineno=109)
           6	LOAD_CONST(arg=1, lineno=109)
           8	BINARY_SUBSCR(arg=None, lineno=109)
          10	STORE_FAST(arg=2, lineno=109)
          12	LOAD_GLOBAL(arg=1, lineno=112)
          14	LOAD_FAST(arg=2, lineno=112)
          16	LOAD_CONST(arg=2, lineno=112)
          18	BINARY_SUBTRACT(arg=None, lineno=112)
          20	CALL_FUNCTION(arg=1, lineno=112)
          22	GET_ITER(arg=None, lineno=112)
>         24	FOR_ITER(arg=75, lineno=112)
          26	STORE_FAST(arg=3, lineno=112)
          28	LOAD_GLOBAL(arg=2, lineno=113)
          30	LOAD_METHOD(arg=3, lineno=113)
          32	LOAD_FAST(arg=0, lineno=113)
          34	LOAD_FAST(arg=3, lineno=113)
          36	LOAD_FAST(arg=3, lineno=113)
          38	LOAD_CONST(arg=2, lineno=113)
          40	BINARY_ADD(arg=None, lineno=113)
          42	LOAD_FAST(arg=2, lineno=113)
 

DEBUG:numba.core.byteflow:dispatch pc=20, inst=CALL_FUNCTION(arg=1, lineno=112)
DEBUG:numba.core.byteflow:stack ['$12load_global.4', '$18binary_subtract.7']
DEBUG:numba.core.byteflow:dispatch pc=22, inst=GET_ITER(arg=None, lineno=112)
DEBUG:numba.core.byteflow:stack ['$20call_function.8']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=24, stack=('$22get_iter.9',), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=24 nstack_initial=1)])
DEBUG:numba.core.byteflow:stack: ['$phi24.0']
DEBUG:numba.core.byteflow:dispatch pc=24, inst=FOR_ITER(arg=75, lineno=112)
DEBUG:numba.core.byteflow:stack ['$phi24.0']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=176, stack=(), blockstack=(), npush=0), Edge(pc=26, stack=('$phi24.0', '$24for_iter.2'), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=176 nstack_initial=0), State(pc_initial=26 nstack_initial=2)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:disp

DEBUG:numba.core.byteflow:stack ['$phi74.0']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=LOAD_FAST(arg=3, lineno=120)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$74load_global.1']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_CONST(arg=2, lineno=120)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$74load_global.1', '$k76.2']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=BINARY_ADD(arg=None, lineno=120)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$74load_global.1', '$k76.2', '$const78.3']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=LOAD_FAST(arg=2, lineno=120)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$74load_global.1', '$80binary_add.4']
DEBUG:numba.core.byteflow:dispatch pc=84, inst=CALL_FUNCTION(arg=2, lineno=120)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$74load_global.1', '$80binary_add.4', '$n82.5']
DEBUG:numba.core.byteflow:dispatch pc=86, inst=GET_ITER(arg=None, lineno=120)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$84call_function.6']
DEBUG:numba.co

DEBUG:numba.core.byteflow:dispatch pc=104, inst=LOAD_FAST(arg=4, lineno=121)
DEBUG:numba.core.byteflow:stack ['$phi90.0', '$phi90.1', '$A92.3', '$98build_tuple.6', '$102binary_subscr.9']
DEBUG:numba.core.byteflow:dispatch pc=106, inst=INPLACE_TRUE_DIVIDE(arg=None, lineno=121)
DEBUG:numba.core.byteflow:stack ['$phi90.0', '$phi90.1', '$A92.3', '$98build_tuple.6', '$102binary_subscr.9', '$scale104.10']
DEBUG:numba.core.byteflow:dispatch pc=108, inst=ROT_THREE(arg=None, lineno=121)
DEBUG:numba.core.byteflow:stack ['$phi90.0', '$phi90.1', '$A92.3', '$98build_tuple.6', '$106inplace_true_divide.11']
DEBUG:numba.core.byteflow:dispatch pc=110, inst=STORE_SUBSCR(arg=None, lineno=121)
DEBUG:numba.core.byteflow:stack ['$phi90.0', '$phi90.1', '$106inplace_true_divide.11', '$A92.3', '$98build_tuple.6']
DEBUG:numba.core.byteflow:dispatch pc=112, inst=LOAD_GLOBAL(arg=1, lineno=123)
DEBUG:numba.core.byteflow:stack ['$phi90.0', '$phi90.1']
DEBUG:numba.core.byteflow:dispatch pc=114, inst=LOAD_FAST(arg=3,

DEBUG:numba.core.byteflow:stack ['$phi224.0', '$phi224.1', '$out226.3', '$k228.4', '$232binary_subscr.7', '$238binary_subscr.10', '$248binary_subscr.15']
DEBUG:numba.core.byteflow:dispatch pc=252, inst=INPLACE_ADD(arg=None, lineno=130)
DEBUG:numba.core.byteflow:stack ['$phi224.0', '$phi224.1', '$out226.3', '$k228.4', '$232binary_subscr.7', '$250binary_multiply.16']
DEBUG:numba.core.byteflow:dispatch pc=254, inst=ROT_THREE(arg=None, lineno=130)
DEBUG:numba.core.byteflow:stack ['$phi224.0', '$phi224.1', '$out226.3', '$k228.4', '$252inplace_add.17']
DEBUG:numba.core.byteflow:dispatch pc=256, inst=STORE_SUBSCR(arg=None, lineno=130)
DEBUG:numba.core.byteflow:stack ['$phi224.0', '$phi224.1', '$252inplace_add.17', '$out226.3', '$k228.4']
DEBUG:numba.core.byteflow:dispatch pc=258, inst=JUMP_ABSOLUTE(arg=112, lineno=130)
DEBUG:numba.core.byteflow:stack ['$phi224.0', '$phi224.1']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=222, stack=('$phi224.0', '$phi224.1'), blockstack=(), npush=0)]
D

...

DEBUG:numba.core.ssa:on stmt: c = freevar(zero: 0.0)
DEBUG:numba.core.ssa:on stmt: $6load_global.1 = global(np: <module 'numpy' from 'E:\\Projetos\\vegetable_oil_mkt\\.env-win-tese\\lib\\site-packages\\numpy\\__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $8load_method.2 = getattr(value=$6load_global.1, attr=nditer)
DEBUG:numba.core.ssa:on stmt: $12call_method.4 = call $8load_method.2(arr, func=$8load_method.2, args=[Var(arr, arraymath.py:171)], kws=(), vararg=None, target=None)
DEBUG:numba.core.ssa:on stmt: $14get_iter.5 = getiter(value=$12call_method.4)
DEBUG:numba.core.ssa:on stmt: $phi16.0 = $14get_iter.5
DEBUG:numba.core.ssa:on stmt: jump 16
DEBUG:numba.core.ssa:==== SSA block analysis pass on 16
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x0000019B9CA5A380>
DEBUG:numba.core.ssa:on stmt: $16for_iter.1 = iternext(value=$phi16.0)
DEBUG:numba.core.ssa:on stmt: $16for_iter.2 = pair_first(value=$16for_iter.1)
DEBUG:numba.core.ssa:on stmt: $16for_iter.3 = pa

DEBUG:numba.core.ssa:find_def var='c' stmt=$28inplace_add.6 = inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=c, rhs=$26call_method.5, static_lhs=Undefined, static_rhs=Undefined)
DEBUG:numba.core.ssa:find_def_from_top label 18
DEBUG:numba.core.ssa:idom 16 from label 18
DEBUG:numba.core.ssa:find_def_from_bottom label 16
DEBUG:numba.core.ssa:find_def_from_top label 16
DEBUG:numba.core.ssa:insert phi node c.2 = phi(incoming_values=[], incoming_blocks=[]) at 16
DEBUG:numba.core.ssa:find_def_from_bottom label 0
DEBUG:numba.core.ssa:incoming_def c = freevar(zero: 0.0)
DEBUG:numba.core.ssa:find_def_from_bottom label 18
DEBUG:numba.core.ssa:incoming_def c.1 = $28inplace_add.6
DEBUG:numba.core.ssa:replaced with: $28inplace_add.6 = inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=c.2, rhs=$26call_method.5, static_lhs=Undefined, static_rhs=Undefined)
DEBUG:numba.core.ssa:on stmt: c.1 = $28inplace_add.6
DEBUG:numba.core.ssa:o

DEBUG:numba.core.byteflow:stack: ['$phi50.0']
DEBUG:numba.core.byteflow:dispatch pc=50, inst=LOAD_GLOBAL(arg=2, lineno=2944)
DEBUG:numba.core.byteflow:stack ['$phi50.0']
DEBUG:numba.core.byteflow:dispatch pc=52, inst=LOAD_CONST(arg=1, lineno=2944)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$50load_global.1']
DEBUG:numba.core.byteflow:dispatch pc=54, inst=CALL_FUNCTION(arg=1, lineno=2944)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$50load_global.1', '$const52.2']
DEBUG:numba.core.byteflow:dispatch pc=56, inst=RAISE_VARARGS(arg=1, lineno=2944)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$54call_function.3']
DEBUG:numba.core.byteflow:end state. edges=[]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=58 nstack_initial=1)])
DEBUG:numba.core.byteflow:stack: ['$phi58.0']
DEBUG:numba.core.byteflow:dispatch pc=58, inst=JUMP_ABSOLUTE(arg=10, lineno=2943)
DEBUG:numba.core.byteflow:stack ['$phi58.0']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=18, stack=('$phi58.0',), b

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x0000019B9CD8FC70>
DEBUG:numba.core.ssa:on stmt: shape = arg(0, name=shape)
DEBUG:numba.core.ssa:on stmt: main_shape = arg(1, name=main_shape)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(len: <built-in function len>)
DEBUG:numba.core.ssa:on stmt: n = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $10load_global.3 = global(range: <class 'range'>)
DEBUG:numba.core.ssa:on stmt: $14call_function.5 = call $10load_global.3(n, func=$10load_global.3, args=[Var(n, arrayobj.py:2941)], kws=(), vararg=None, target=None)
DEBUG:numba.core.ssa:on stmt: $16get_iter.6 = getiter(value=$14call_function.5)
DEBUG:numba.core.ssa:on stmt: $phi18.0 = $16get_iter.6
DEBUG:numba.core.ssa:on stmt: jump 18
DEBUG:numba.core.ssa:==== SSA block analysis pass on 18
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x0000019B9CD8FC70>
DEBUG:numba.core.ssa:on s

array([[0.71428571, 0.14285714, 0.14285714]])

In [96]:
neg_array = grid_neg.to_numpy()
pos_array = grid_pos.to_numpy()

pos_array

array([[0.96260939, 0.        , 0.        , ..., 0.        , 0.        ,
        0.03739061],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.39269406, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.6056338 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.4       , 0.        ,
        0.6       ]])

In [97]:
mc = qe.MarkovChain(pos_array)
mc.stationary_distributions 

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])

In [98]:
mc.is_irreducible 

False

In [99]:
mc

Markov chain with transition matrix 
P = 
[[0.96260939 0.         0.         ... 0.         0.         0.03739061]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.39269406 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.6056338  0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.4        0.         0.6       ]]
and stationary distributions 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

In [ ]:
pos_array[1]

In [ ]:
P = np.array(pos_array)

ψ = pos_array[1] #(0.0, 0.2, 0.8)        # Initial condition

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.set(xlim=(0, 1), ylim=(0, 1), zlim=(0, 1),
       xticks=(0.25, 0.5, 0.75),
       yticks=(0.25, 0.5, 0.75),
       zticks=(0.25, 0.5, 0.75))

x_vals, y_vals, z_vals = [], [], []
for t in range(20):
    x_vals.append(ψ[0])
    y_vals.append(ψ[1])
    z_vals.append(ψ[2])
    ψ = ψ @ P

ax.scatter(x_vals, y_vals, z_vals, c='r', s=60)
ax.view_init(30, 210)

mc = qe.MarkovChain(P)
ψ_star = mc.stationary_distributions[0]
ax.scatter(ψ_star[0], ψ_star[1], ψ_star[2], c='k', s=60)

plt.show()